In [ ]:
%matplotlib ipympl
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import numpy as np
import glob
import pickle 
from modules import LatentLowRankRNN, train
from RNN_helpers import plot_field_noscalings, load_data, plot_trajectories
import yaml
import matplotlib.pyplot as plt

In [ ]:
params = yaml.safe_load(open('../data_parameters.yml','r'))

In [ ]:
files = glob.glob(params['out_folder'] + '/*.pkl')
data_file = [f for f in files if params['session'] in f and 'avg' not in f and 'maze' in f and '-99' not in f]
data_file = np.sort(data_file)
pipeline = pickle.load(open(os.path.join(params['out_folder'], params['session'] + '_PCA_3dim.pkl'),'rb'))

x_train, dx_train, label_train = load_data(data_file,
                                           params, 
                                           mazes=[2],
                                           paths=[1],
                                           pipeline=pipeline, 
                                           smooth_window=params['smooth_window'], 
                                           interval=params['sampling_interval'])

In [ ]:
noise_std = 5e-2
alpha = 0.2
rank = 2

In [ ]:
input_size = x_train.shape[-1]
net = LatentLowRankRNN(input_size, [16,32,512], rank=rank, alpha=alpha, noise_std=noise_std)
train(net, x_train, epochs=500, batch_size=32, lr=0.01)

In [ ]:
x_ = x_train[:,[0],:].to(device=net.m.device)
x, _, _ = net(x_, seq_len=x_train.shape[1])
plt.figure()
plt.plot(x[0].detach().cpu().numpy(), 'g--')
plt.plot(x_train[0],'r')
plt.plot()

In [ ]:
ax, _ = plot_field_noscalings(net, input=None, xmin=-5, xmax=5, ymin=-5, ymax=5, add_fixed_points=False)

_, _, z = net(x_train[0][0:2].unsqueeze(1).to(device=net.m.device))
plot_trajectories(net, z.detach().cpu().numpy() , ax, n_traj=2, style="-", c="C0")